<a href="https://colab.research.google.com/github/diwaskunwar10/mistral/blob/main/mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
model_name = 'filipealmeida/Mistral-7B-Instruct-v0.1-sharded'


In [3]:
import torch
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

    return model

def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer


In [4]:
model = load_quantized_model(model_name)

tokenizer = initialize_tokenizer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [5]:
stop_token_ids = [0]

text = "write an essay on capital city of nepal in 300 words"

encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_input = encoded
generated_ids = model.generate(**model_input, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1455: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


write an essay on capital city of nepal in 300 words.
Nepal's capital city, Kathmandu, is a bustling metropolis that serves as the political, cultural, and economic center of the country. It is the largest city in Nepal, with a population of over two million people.

Kathmandu is located in the Central Region of Nepal and is surrounded by snow-capped mountains. The city is known for its rich cultural heritage, with numerous temples, monasteries, and historical sites that showcase the country's unique blend of Hinduism and Buddhism. Some of the most famous temples in Kathmandu include Pashupatinath Temple, Boudhanath Stupa, and Swayambhunath Temple.

The city also boasts a thriving tourism industry, with many people visiting Kathmandu each year to explore its beautiful architecture, stunning views, and traditional Nepalese culture. The city is home to several international airlines, making it easy for visitors to reach their destination.

Despite its rich cultural heritage and thriving 

In [7]:
stop_token_ids = [0]
text = "write an essay on capital city of nepal in 300 words"

num_epochs = 3

# Split text into smaller chunks
chunks = [text[i:i+200] for i in range(0, len(text), 200)]  # Split into chunks of 200 characters

generated_texts = []
for _ in range(num_epochs):
    for chunk in chunks:
        encoded = tokenizer(chunk, return_tensors="pt", add_special_tokens=False)
        generated_ids = model.generate(**encoded, max_length=2000, do_sample=True)
        decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        generated_texts.extend(decoded)

# Join generated texts
final_generated_text = " ".join(generated_texts)
print(final_generated_text)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


write an essay on capital city of nepal in 300 words, but i am not able to write.

i cannot seem to find any information on the internet or even books to help me write an essay on the capital city of nepal. Please help me. Any information will be appreciated.

[Your Name] write an essay on capital city of nepal in 300 words.?

My essay on the capital city of Nepal, Kathmandu, will focus on its various cultural and historical landmarks that attract tourists from all over the world. Kathmandu is a city located in the central region of Nepal, and it has a rich history dating back to ancient times.

One of the most iconic landmarks in Kathmandu is the Pashupatinath Temple, which is a UNESCO World Heritage Site. The temple is dedicated to the Hindu god Shiva, and it is known for its stunning architecture and intricate carvings. The temple is surrounded by beautiful gardens and rivers, making it a popular spot for visitors.

Another must-see landmark in Kathmandu is the Swayambhunath Temple.